# NLP Individual CourseWork 
## Name-Vipul Mishra
## URN-6657466
## Topic- Toxic Comment Classifier
Problem Statment - In this Experiment, the challenge is to build a multilabeled model which will detect variety of toxic commments like threats, obscenity, insults, and identity-based hate. I’ll be using a dataset of comments which i have taken from kaggle. Implementing and Improvements to the current model will hopefully help online discussion become more productive,respectful and less toxic.

##Importing Required Libraries

In [1]:
#import libraries
import pandas as pd
import os    #Operating System Library
import nltk  #natural language Library
import re    #Regular Expression Library
import gensim ##Tokenization Library
import spacy  ##Lemmatize
from gensim.utils import simple_preprocess
import numpy as np 
import time 
import string 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix
from wordcloud import WordCloud,STOPWORDS
from nltk.corpus import stopwords
nltk.download('stopwords')

ImportError: cannot import name 'joblib' from 'sklearn.externals' (/opt/anaconda3/lib/python3.8/site-packages/sklearn/externals/__init__.py)

# Loading DataSet and Data Analysis

In [ ]:
#Read data into train_df 
train = pd.read_csv("train.csv")

#Printing Head to view 
train.head()

In [ ]:
rowSums = train.iloc[:,2:].sum(axis=1)
clean_count = (rowSums==0).sum(axis=0)

total=len(train)
clean_comments=clean_count
comment_labels=total-clean_comments

print("Total number of comments = ",total)
print("Comments without label= ",clean_comments)
print("Comments with labels =",comment_labels)

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
#Checking for missing values or not 
train.isnull().any()

##Data Visualization


In [ ]:
### Bar Plot 
categories = list(train.columns.values)
categories = categories[2:]
print(categories)
sns.set(font_scale = 2)
plt.figure(figsize=(15,8))

ax= sns.barplot(categories, train.iloc[:,2:].sum().values)

plt.title("Total Number of Comments in Each Label", fontsize=24)
plt.ylabel('Number of comments', fontsize=18)
plt.xlabel('Label Type ', fontsize=18)

#adding the text labels
rects = ax.patches
labels = train.iloc[:,2:].sum().values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=18)

plt.show()

From the above the bar plot we can clearly see that the current dataset we have is unbalanced as most of the comments fall under 'Toxic' Label.
We can have one more experimental setup by dropping the 'Toxic' Label which will make the dataset more balanced. 

In [ ]:
counts = []
for category in categories:
    counts.append((category, train [category].sum()))
df_stats = pd.DataFrame(counts, columns=['category', 'number of comments'])
df_stats

In [ ]:
Sums = train.iloc[:,2:].sum(axis=1)
multiLabel_counts = Sums.value_counts()
multiLabel_counts = multiLabel_counts.iloc[1:]

sns.set(font_scale = 2)
plt.figure(figsize=(15,8))

ax = sns.barplot(multiLabel_counts.index, multiLabel_counts.values)

plt.title("Comments having multiple labels ")
plt.ylabel('Number of comments', fontsize=18)
plt.xlabel('Number of labels', fontsize=18)

rects = ax.patches
labels = multiLabel_counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

'Comments Having multiple comments' Bar Graph shows that we have very less comments which falls under all 6 Labels and majority of the comments falls under any of the one label only.   

In [ ]:
# Word Cloud for test set

words = ' '.join([text for text in train['comment_text'] ])


word_cloud = WordCloud(
                       width=1600,
                       height=800,
                       #colormap='PuRd', 
                       margin=0,
                       max_words=500,
                       max_font_size=150, min_font_size=30,  # Font size range
                       background_color="white").generate(words)

plt.figure(figsize=(10, 16))
plt.imshow(word_cloud, interpolation="bilinear")
plt.title('WordCloud', fontsize = 40)
plt.axis("off")
plt.show()

Here we have generated a word cloud from the different words mentioned in the 'comment_text' field.

In [ ]:
target_data = train.drop(['id', 'comment_text'], axis=1)
corrMatrix = target_data.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

Above generated Heat-map shows the relation between the different labels. For example in this case we have 'severe_toxic' and 'toxic' relationship as 0.31.

In [ ]:
import nltk
from nltk.tokenize import ToktokTokenizer
x=nltk.FreqDist(ToktokTokenizer().tokenize(train['comment_text']))
plt.figure(figsize=(16,5))
x.plot(30)

Generating Word Frequency Graph from the Pre-Processed DataSet.
It shows the frequency of each word throughout the dataset. 

In [ ]:
#Sampling the data
train = train.sample(1000)
train.head()

In [ ]:
train_labels = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
train_labels.head()

#Data Pre-Processing


In [ ]:
 #Defining function for removing punctuation,lowercase, removing hyperlinks
 def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
# Applying the clean_text on train set

train['comment_text_cleaned'] = train['comment_text'].apply(lambda x: clean_text(x))


In [ ]:
train.head()

Tokenizing using gensim

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = train.comment_text_cleaned.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

In [ ]:
X= []
sentences = list(train["comment_text_cleaned"])
for texts in sentences:
    X.append(clean_text(texts))

y = train_labels.values


After cleaning the data and converting them into tokens, we are now converting the cleaned tokens into a list from which we will remove all the stops words, generated bigrams and trigrams, lemmatize the data.

In [ ]:
%%time
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100) 

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
#Functions for StopWords, n-grams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [ ]:
stop_words = stopwords.words('english')
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:5][0][:30])

De-Tokenizing for further Vectorization


In [ ]:
# de-tokenization
detokenized_doc = []
clean_df=pd.DataFrame()
for i in range(len(data_lemmatized)):
    t = ' '.join(data_lemmatized[i])
    detokenized_doc.append(t)

clean_df= detokenized_doc
#clean_df=list(clean_df.list()
clean_df

###Splitting the Data in Train and Test  
Splitting is done using 70/30 split as it fairly splits the data into enough train data and test data which further gives us the more precision(accuracy) while testing the model. If the data is splitted as per 75/25 ratio then surely we would have trained our model with more than required training data but then it would have impacted our volume of test data.  

In [ ]:
#Splitting the data
X_train, X_test, y_train, y_test = train_test_split(clean_df,train_labels, test_size=0.30, random_state=42)


# Experiment Setup 1

> In this experimental setup i have opted for CountVector for feature vectorization and then implemented the Binary Relevance Model and found out the accuracy and other metrices




Vectorizing using CountVector


In [ ]:
##Using CountVectorizer to Vectorize
from sklearn.feature_extraction.text import CountVectorizer

# List of text documents
vectorizer = CountVectorizer()

# Tokenize and build vocab

vectorizer.fit(X_train)
print("Vocab for X_train:")
print(vectorizer.vocabulary_)

vectorizer.fit(X_test)
print("Vocab for X_test:")
print(vectorizer.vocabulary_)

# Encode document
cv_x_train = vectorizer.transform(X_train)
cv_x_test = vectorizer.transform(X_test)


Implementing Binary Relevance Model on features using CountVector method

In [ ]:
#Install required packages 
!pip install Cython
!pip install scikit-multilearn

In [ ]:
##### Binary Relevance(GaussianNB)using CountVector 
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(cv_x_train, y_train)
# predict
predictions_br_cv = classifier.predict(cv_x_test)
# accuracy
accuracy_br_cv = accuracy_score(y_test,predictions_br_cv)
print("Accuracy = ",accuracy_br_cv)

In [ ]:
#F1 Score Binary Relevance
f1=f1_score(y_test, predictions_br_cv,average=None)
f1

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_br_cv)

# Experiment Setup 2

> Here I have opted for TF-IDFVectorizer for feature vectorization and generated N-grams by altering the hyperparameters in TFIDFVectorizer, further i have implemented Binary Relevance Model(GaussianNB) for each of the n-gram (1-gram,2-gram,3-gram) and identified the accuracy for each model.





## Vectorizing using TF-IDF Vectors

In [ ]:
#Vectorizing using Tf-Idf Uni-gram
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',max_features=40000, ngram_range=(1,1))
vectorizer.fit(X_train)
X_train_uni = vectorizer.transform(X_train)
X_test_uni = vectorizer.transform(X_test)

In [ ]:
#Vectorizing using Tf-Idf Bi-gram
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',max_features=40000, ngram_range=(1,2))
vectorizer.fit(X_train)
X_train_bi = vectorizer.transform(X_train)
X_test_bi = vectorizer.transform(X_test)

In [ ]:
#Vectorizing using Tf-Idf Tri-gram
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',max_features=40000, ngram_range=(1,2))
vectorizer.fit(X_train)
X_train_tri = vectorizer.transform(X_train)
X_test_tri = vectorizer.transform(X_test)

# Binary Relevance Model
Implementing using TF-IDF Vectorizer for 1-gram,2-gram and 3-gram

Uni-Gram

In [ ]:
##### using binary relevance with unigram
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train_uni, y_train)
# predict
predictions_br_uni = classifier.predict(X_test_uni)
# accuracy
accuracy_br_uni = accuracy_score(y_test,predictions_br_uni)
print("Accuracy = ",accuracy_br_uni)

In [ ]:
#F1 Score Binary Relevance TF-IDF Unigram
f1_score(y_test, predictions_br_uni,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_br_uni)

Bi-Gram

In [ ]:
##### using binary relevance with bigram
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train_bi, y_train)
# predict
predictions_br_bi = classifier.predict(X_test_bi)
# accuracy
accuracy_br_bi = accuracy_score(y_test,predictions_br_uni)
print("Accuracy = ",accuracy_br_bi)

In [ ]:
##F1 Score Binary Relevance-bigram
f1_score(y_test, predictions_br_bi,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_br_bi)

Tri-gram

In [ ]:
##### using binary relevance with trigram
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(X_train_tri, y_train)
# predict
predictions_br_tri = classifier.predict(X_test_tri)
# accuracy
accuracy_br_tri = accuracy_score(y_test,predictions_br_tri)
print("Accuracy = ",accuracy_br_tri)

In [ ]:
#F1 Score Binary Relevance-trigram
f1_score(y_test, predictions_br_tri,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_br_bi)

From the accuracy score of 1-gram, 2-gram and 3-gram, we can easily say that 1-gram and 2-gram gives same accuracy.

#Experiment Setup 3 


> In experiment 3, i have used Classifier Chains model using different Algorithm such as SVC, Logistic Regression and RandomForest Classifier. 
Also, identified the accuracy score and other metrices for each of the models implemented for comparison. 






# Classifier Chains Model, F1 Score and Confusion Matrix
Implemented Using different classifer models SVC,Logistic Regression, RandomForestClassifier

Classifier Chain with SVC

In [ ]:
#### Using Classifier Chains with SVC
from sklearn.svm import SVC
from skmultilearn.problem_transform import ClassifierChain
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(SVC())
# Training logistic regression model on train data
classifier.fit(X_train_uni, y_train)
# predict
predictions_cc_uni = classifier.predict(X_test_uni)
# accuracy
accuracy_cc_svc = accuracy_score(y_test,predictions_cc_uni)
print("Accuracy = ",accuracy_cc_svc)

In [ ]:
#F1 Score Classifier Chains SVC
f1_score(y_test, predictions_cc_uni,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_cc_uni)

Classifier Chain with Logistic Regression

In [ ]:
#### Using Classifier Chains with LogisticRegression

from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_uni, y_train)
# predict
predictions_cc_lr = classifier.predict(X_test_uni)
# accuracy
accuracy_cc_lr = accuracy_score(y_test,predictions_cc_lr)
print("Accuracy = ",accuracy_cc_lr)

In [ ]:
#F1 Score Classifier Chains Logistic Regression
f1_score(y_test, predictions_cc_uni,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_cc_lr)

Classifier Chain with RandomForestClassifier

In [ ]:
#### Using Classifier Chains with RandomForest

from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(RandomForestClassifier(n_estimators=100))
# Training logistic regression model on train data
classifier.fit(X_train_uni, y_train)
# predict
predictions_cc_rfc = classifier.predict(X_test_uni)
# accuracy
accuracy_cc_rfc = accuracy_score(y_test,predictions_cc_rfc)
print("Accuracy = ",accuracy_cc_rfc)

In [ ]:
#F1 Score Classifier Chains RandomForest
f1_score(y_test, predictions_cc_rfc,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_cc_lr)

# Label Power Set Model, F1 Score and Confusion Matrix
Implementing using GaussianNB for 1-gram and 2-gram

Uni-gram

In [ ]:
# Using Label Powerset Uni-gram
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(GaussianNB())
# train
classifier.fit(X_train_uni, y_train)
# predict
predictions_lp_ga_uni = classifier.predict(X_test_uni)
# accuracy
accuracy_lp_ga_uni = accuracy_score(y_test,predictions_lp_ga_uni)
print("Accuracy = ",accuracy_lp_ga_uni)

In [ ]:
#F1 Score Label Powerset unigram
f1_score(y_test, predictions_lp_ga_uni,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_lp_ga_uni)

Bi-Gram

In [ ]:
# Using Label Powerset Bi-gram
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(GaussianNB())
# train
classifier.fit(X_train_bi, y_train)
# predict
predictions_lp_ga_bi = classifier.predict(X_test_bi)
# accuracy
accuracy_lp_ga_bi = accuracy_score(y_test,predictions_lp_ga_bi)
print("Accuracy = ",accuracy_lp_ga_bi)

In [ ]:
#F1 Score Label Powerset unigram
f1_score(y_test, predictions_lp_ga_bi,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_lp_ga_bi)

#Experiment Setup 4

> In setup 4 i have used Adapted Algorithm model using MLkNN Algorithm with different 'K' values to give accuracy score and then compare the two to find out the better one.



# Adapted Algorithm Model, F1 Score and Confusion Matrix

K=5

In [ ]:
###Using Adapted Algo for Unigram

from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=5)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train_bi).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test_bi).toarray()
# train
classifier_new.fit(x_train, y_train)
# predict
predictions_aa_mlknn_bi = classifier_new.predict(x_test)
# accuracy
accuracy_aa_mlknn_uni = accuracy_score(y_test,predictions_aa_mlknn_bi)
print("Accuracy = ",accuracy_aa_mlknn_bi)

In [ ]:
#F1 Score Adopted Algo unigram
f1_score(y_test, predictions_aa_mlknn_uni,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_aa_mlknn_uni)

K=10

In [ ]:
###Using Adapted Algo for Bi-gram

from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train_bi).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test_bi).toarray()
# train
classifier_new.fit(x_train, y_train)
# predict
predictions_aa_mlknn_bi = classifier_new.predict(x_test)
# accuracy
accuracy_aa_mlknn_bi = accuracy_score(y_test,predictions_aa_mlknn_bi)
print("Accuracy = ",accuracy_aa_mlknn_bi)

In [ ]:
#F1 Score Adopted Algo Bi-gram
f1_score(y_test, predictions_aa_mlknn_uni,average=None)

In [ ]:
##Making Confusion Matrix for the model
multilabel_confusion_matrix(y_test, predictions_aa_mlknn_uni)

From either K=5 or K=10, we have acheived same accuracy, making the change irrelevant. 

# Comparing Accuracy Achieved from using different Models and different Experimental Setup

In [ ]:
print("Accuracy for Count Vector = ",accuracy_br_cv)
print(" ")
print("Accuracy for BR Models(1-gram) = ",accuracy_br_uni)
print("Accuracy for BR Models(2-gram) = ",accuracy_br_bi)
print("Accuracy for BR Models(3-gram) = ",accuracy_br_tri)
print(" ")
print("Accuracy for ClassifierChain(SVC) = ",accuracy_cc_svc)
print("Accuracy for ClassifierChain(lr) = ",accuracy_cc_lr)
print("Accuracy for ClassifierChain(rfc) = ",accuracy_cc_rfc)
print(" ")
print("Accuracy for LabelPowerset(Gaussian_1-gram) = ",accuracy_lp_ga_uni)
print("Accuracy for LabelPowerset(Gaussian_2-gram) = ",accuracy_lp_ga_bi)
print(" ")
print("Accuracy for AdaptedAlgo(MLKnn_1-gram) = ",accuracy_aa_mlknn_uni)
print("Accuracy for AdaptedAlgo(MLKnn_2-gram) = ",accuracy_aa_mlknn_bi)
print(" ")

After performing all the different experiments using different Feature Vectorization and using different ML models with different algorithms and by analysing the accuracy score for each experiment setup, ClassifierChain Model has turned out to be the better choice for implementing and addressing the multi-label problem at hand. 
Although the accuracy score from different setups is not varying much in some case but the F1 Score achieved is quite low, this arises the need for retraining the model using different parameters or algorithm altogther. 
Further experiments can be done using different models other than the one used in above setups. 

By doing multiple different experiments to acheive best accuracy, I have learnt so many ways for implementing an NLP Model. Experimenting with variety of setups helps getting familiar with this NLP module and opens up the possibility for further evaluation and learning. This experiment has helped me in upscaling my critical thinking skill which will be an added advantage in forthcoming ML Models.